In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models
from tensorflow.keras.saving import register_keras_serializable
import tensorflow as tf
import joblib

# === 1. Load Dataset ===
data = pd.read_csv("dataset/tornado_data.csv")
X = data.drop("tornado_binary", axis=1).astype("float32")

def generate_trust_score(row):
    srh   = row["storm_relative_helicity"]
    cape  = row["CAPE"]
    lcl   = row["lifted_condensation_level"]
    shear = row["bulk_wind_shear"]
    stp   = row["significant_tornado_param"]

    srh_factor   = np.clip((srh - 150) / 250, 0, 1)
    cape_factor  = np.clip((cape - 1000) / 2500, 0, 1)
    lcl_factor   = np.clip((1500 - lcl) / 1500, 0, 1)
    shear_factor = np.clip((shear - 8) / 20, 0, 1)
    stp_factor   = np.clip((stp - 0.5) / 2.5, 0, 1)

    composite = (
        0.25 * srh_factor +
        0.25 * cape_factor +
        0.2  * lcl_factor +
        0.15 * shear_factor +
        0.15 * stp_factor
    )

    return 0.5 + composite

trust_labels = X.apply(generate_trust_score, axis=1).astype("float32")

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, "models/TornadoTrustScaler.pkl")

@register_keras_serializable()
def trust_activation(x):
    return 0.5 + tf.sigmoid(x)

input_layer = layers.Input(shape=(5,))
x = layers.Dense(32, activation="relu")(input_layer)
x = layers.Dense(16, activation="relu")(x)
x = layers.Dense(8, activation="relu")(x)
trust_output = layers.Dense(1, activation=trust_activation)(x)

model = models.Model(inputs=input_layer, outputs=trust_output)
model.compile(optimizer="adam", loss="mse")
model.summary()

# === 6. Train ===
model.fit(X_scaled, trust_labels, validation_split=0.2, batch_size=32, epochs=10)

import tf2onnx
import json
model.save("models/TornadoTrustNet.h5")
model_proto, _ = tf2onnx.convert.from_keras(model)
with open(f"models/ONNX/TornadoTrustNet.onnx", "wb") as f:
    f.write(model_proto.SerializeToString())

scaler_data = {
        "mean": scaler.mean_.tolist(),
        "scale": scaler.scale_.tolist()
}
with open(f"models/SCALER-CFG/TornadoScaler.json", "w") as f:
    json.dump(scaler_data, f)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 5)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 865 (3.38 KB)

 Trainable params: 865 (3.38 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 0.0357 - val_loss: 0.0012
Epoch 2/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0010 - val_loss: 5.0164e-04
Epoch 3/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.5564e-04 - val_loss: 3.1512e-04
Epoch 4/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.6671e-04 - val_loss: 2.2191e-04
Epoch 5/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.0189e-04 - val_loss: 1.8417e-04
Epoch 6/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.6267e-04 - val_loss: 1.5848e-04
Epoch 7/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.4438e-04 - val_loss: 1.4600e-04
Epoch 8/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.2357e-04 - val_loss: 1.3183e-04
Epoch 9/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1662e-04 - val_loss: 1.2534e-04
Epoch 10/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.1437e-04 - val_loss: 1.1747e-04




ERROR:tf2onnx.tfonnx:rewriter <function rewrite_constant_fold at 0x000002075AA3A2A0>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.
